In [1]:
from tensorflow import keras
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv('datasets/train.csv')
train

,Unnamed: 0,image_url,humour,sarcastic,offensive,motivational,overall,ocr
0,0,https://encrypted-tbn0.gstatic.com/images?q=tb...,very_funny,twisted_meaning,not_offensive,not_motivational,neutral,Relationship status.\nSingle\nMarried\nIt's co...
1,1,https://encrypted-tbn0.gstatic.com/images?q=tb...,very_funny,twisted_meaning,slight,not_motivational,neutral,Bernie or REAGAN?\nBe informed. Compare them o...
2,2,https://encrypted-tbn0.gstatic.com/images?q=tb...,funny,general,not_offensive,not_motivational,positive,I HEAR SOMETHING\nBetter run a mile\nOMG RAJ?\...
3,3,https://preview.redd.it/iwcz3o2niix61.jpg?widt...,very_funny,twisted_meaning,not_offensive,not_motivational,positive,Redditors Visiting IG memepages\n*dies from cr...
4,4,https://encrypted-tbn0.gstatic.com/images?q=tb...,not_funny,not_sarcastic,very_offensive,not_motivational,negative,"Samajh nahi aaya Par sun ke acabo,\nBUDGET SPE..."
...,...,...,...,...,...,...,...,...
6995,6995,https://preview.redd.it/3c7iiijlg9d61.jpg?widt...,hilarious,general,slight,not_motivational,negative,$14 y/o nibbi*\n*16 y/o nibba*\n*20 y/o single...
6996,6996,https://encrypted-tbn0.gstatic.com/images?q=tb...,funny,very_twisted,hateful_offensive,not_motivational,negative,I DON'T HAVE A PROBLEM WITH\nIMMIGRATION\nIF W...
6997,6997,https://encrypted-tbn0.gstatic.com/images?q=tb...,hilarious,not_sarcastic,not_offensive,not_motivational,positive,"Como umbalin\nHey Alia,\nWhat is the\nOpposite..."
6998,6998,https://encrypted-tbn0.gstatic.com/images?q=tb...,hilarious,general,very_offensive,motivational,negative,baby kya kr rhe ho?\ntumhara favorite football...


In [5]:
train.isnull().sum()

Unnamed: 0      0
image_url       0
humour          0
sarcastic       0
offensive       0
motivational    0
overall         0
ocr             0
dtype: int64

In [11]:
import cv2
def read_image(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(224,224))
    return img

In [14]:
img1=read_image(os.path.join('.','datasets','trainimages','9.jpg'))

In [15]:
img1.shape

(224, 224, 3)

In [20]:
X=[]
for i in range(3000):
    img=read_image(os.path.join('.','datasets','trainimages',str(i)+'.jpg'))
    X.append(img)
    

In [23]:
X=np.array(X)
X=X/255

In [24]:
X

array([[[[0.99607843, 0.99607843, 0.99607843],
         [0.96078431, 0.96078431, 0.96078431],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [0.82352941, 0.82352941, 0.82352941]],

        [[1.        , 1.        , 1.        ],
         [0.98823529, 0.98823529, 0.98823529],
         [0.94509804, 0.94509804, 0.94509804],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [0.81960784, 0.81960784, 0.81960784]],

        [[0.98039216, 0.98039216, 0.98039216],
         [0.99215686, 0.99215686, 0.99215686],
         [0.98039216, 0.98039216, 0.98039216],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [0.82352941, 0.82352941, 0.82352941]],

        ...,

        [[0.99607843, 0.99607843, 0.99607843],
         [0.99607843, 0.99607843, 0.99607843]

In [28]:
train['overall'].value_counts()

overall
neutral          2970
positive         1902
negative         1454
very_positive     373
very_negative     301
Name: count, dtype: int64

In [43]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(train['overall'])

In [44]:
y=pd.Series(y)
y.value_counts()

1    2970
2    1902
0    1454
4     373
3     301
Name: count, dtype: int64

In [45]:
len(y.unique())

5

In [35]:
cnn= keras.models.Sequential([
    keras.layers.Conv2D(64,(3,3),input_shape=(224,224,3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(128,(3,3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64,activation='relu'),
    keras.layers.Dense(len(y.unique()),activation='softmax')

])

c:\Users\Kaila\.pyenv\pyenv-win\versions\3.10.0\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
cnn.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [46]:

y=y[:3000]

In [47]:
len(y)

3000

In [50]:
cnn.fit(X,y,epochs=3)

Epoch 1/3
20/94 ━━━━━━━━━━━━━━━━━━━━ 2:04 2s/step - accuracy: 0.9981 - loss: 0.0287

KeyboardInterrupt: 

In [51]:
cnn.predict(X)

94/94 ━━━━━━━━━━━━━━━━━━━━ 34s 354ms/step


array([[2.6341137e-05, 9.9997294e-01, 7.6255139e-07, 3.3354203e-12,
        2.8965386e-11],
       [6.7739420e-10, 9.9866414e-01, 1.3358289e-03, 2.2547562e-08,
        2.5243507e-11],
       [1.4319552e-04, 2.0162354e-03, 9.9781537e-01, 2.2434953e-05,
        2.6958667e-06],
       ...,
       [2.2393088e-04, 9.9890459e-01, 2.9163259e-06, 8.6858938e-04,
        3.7259131e-08],
       [3.0847008e-03, 2.4456859e-03, 9.9446684e-01, 3.4979817e-07,
        2.5034533e-06],
       [5.2636897e-04, 9.9947363e-01, 7.7567779e-11, 2.6210133e-17,
        8.0817091e-15]], shape=(3000, 5), dtype=float32)

In [52]:
test=[]
for i in range(500):
    img=read_image(os.path.join('.','datasets','testimages',str(i)+'.jpg'))
    test.append(img)